In [2]:
import os
import glob
import pandas as pd
import geopandas as gpd
import rasterstats as rs

###Directories
maindir = "C:/Users/mccoo/OneDrive/mcook/"
buprDir = os.path.join(maindir, 'data', 'hisdac_us', 'BUPR')
print(buprDir)

###List out the BUPR raster paths
rasters = glob.glob(buprDir+"/*.tif", recursive=True)
print(rasters)

###Read in the joined event shapefile (ICS-FIRED)
evt_file = os.path.join(maindir, 'ics209', 'data', 'ics-fired', 'ics-fired_to2020.gpkg')
evt = gpd.read_file(evt_file, driver="GPKG")  

C:/Users/mccoo/OneDrive/mcook/data\hisdac_us\BUPR
['C:/Users/mccoo/OneDrive/mcook/data\\hisdac_us\\BUPR\\BUPR_2000.tif', 'C:/Users/mccoo/OneDrive/mcook/data\\hisdac_us\\BUPR\\BUPR_2005.tif', 'C:/Users/mccoo/OneDrive/mcook/data\\hisdac_us\\BUPR\\BUPR_2010.tif', 'C:/Users/mccoo/OneDrive/mcook/data\\hisdac_us\\BUPR\\BUPR_2015.tif']


C:\Users\mccoo\miniconda3\envs\mcenv\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [8]:
# ##Create buffer distance list
# buffers = list([0, 250, 1000, 4000])
# buf_out = []
# for i in range(0, len(buffers)):
    
#     if buffers[i]==0:
#         print("Event-level (no buffer) ...")
#         evt=evt
#         buf_out.append(evt)
#     else:
#         print("Event buffer at distance: "+str(buffers[i])+" meters")
#         evt['geometry'] = evt.geometry.buffer(int(buffers[i]))
#         buf_out.append(evt)

# print(len(buf_out))

# evt['geometry'] = evt.geometry.buffer(250)

In [9]:
###Define zonal statistics function
def get_zonal_stats(vector, raster, stats):
    # Run zonal statistics, store result in geopandas dataframe
    result = rs.zonal_stats(vector, raster, stats=stats, geojson_out=True)
    return gpd.GeoDataFrame.from_features(result) 

In [3]:
###Get the years and filter the geodataframe
bupr_years = [int(os.path.basename(r)[5:9]) for r in rasters]
print(bupr_years)

evt_years = list(evt['ig_year'].unique())
for i in range(0, len(evt_years)):
    evt_years[i] = int(evt_years[i])
print("min burn year: "+str(min(evt_years)))
print("max burn year: "+str(max(evt_years)))

[2000, 2005, 2010, 2015]
min burn year: 2000
max burn year: 2020


In [11]:
###Now, perform zonal statistics for each semi-decade
bupr_sums = [] #Empty list for results    
for yr in bupr_years:

    bupr = os.path.join(buprDir, 'BUPR_'+str(yr)+'.tif')

    if yr == 2015:

        polys = evt[(evt['ig_year'] >= yr) & (evt['ig_year'] <= max(evt_years))]
        print(list(polys['ig_year'].unique()))

        zs = get_zonal_stats(polys, bupr, stats='max median')
        
        zs = zs.rename(columns={'max': 'bupr_max', 'median': 'bupr_med'})

        bupr_sums.append(zs)

    else:

        polys = evt[(evt['ig_year'] >= yr) & (evt['ig_year'] < yr+5)]
        print(list(polys['ig_year'].unique()))

        zs = get_zonal_stats(polys, bupr, stats='max median')

        zs = zs.rename(columns={'max': 'bupr_max', 'median': 'bupr_med'})

        bupr_sums.append(zs)

print(len(bupr_sums))

[2000.0, 2001.0, 2002.0, 2003.0, 2004.0]
[2005.0, 2006.0, 2007.0, 2008.0, 2009.0]
[2010.0, 2011.0, 2012.0, 2013.0, 2014.0]
[2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0]
4


In [12]:
###Merge results into one dataframe
# Combine the data frames
print("Merging data frames ...")

gdf = pd.concat(bupr_sums)
print(type(gdf))

# Write to file
print("Writing output file ...")
out_file = os.path.join(maindir, 'FIRED', 'data', 'update0621', 'fired_to2021091_events_albers_conus_bupr_MaxMed.shp')
gdf.to_file(out_file)

Merging data frames ...
<class 'geopandas.geodataframe.GeoDataFrame'>
Writing output file ...
